In [ ]:
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import time
import textwrap
from IPython.display import Markdown, display

In [ ]:
genai.configure(api_key="Enter_your_Api_here")


In [ ]:
def wrap_text(text, width=100):
    return '\n'.join(textwrap.wrap(text, width=width))

In [ ]:
def type_text(text, delay=0.005, width=100):
    wrapped = wrap_text(text, width)
    for line in wrapped.split('\n'):
        for char in line:
            print(char, end='', flush=True)
            time.sleep(delay)
        print()

In [ ]:
def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    for tag in soup(["script", "style", "img", "video", "iframe"]):
        tag.decompose()

    text = soup.get_text(separator="\n")
    cleaned_text = ' '.join(text.split())
    return cleaned_text

In [ ]:
def summarize_with_gemini(text):
    model = genai.GenerativeModel("models/gemini-1.5-flash")
    prompt = f"""
Please generate a **detailed summary** of the following webpage content:
- Use bullet points wherever appropriate.
- Include subheadings like 'Overview', 'Key Insights', etc.
- Do not mention any images, videos, or links.

Text content:
{text}
"""
    response = model.generate_content(prompt)
    return response.text

In [ ]:
def pretty_display(summary, url, use_typing=True):
    heading_main = "📄 SUMMARY OF THE PAGE"
    source_line = f"🔗 URL Entered: {url}"

    if use_typing:
        type_text(heading_main)
        type_text(source_line)
    else:
        display(Markdown(f"## {heading_main}"))
        display(Markdown(f"**URL Entered:** {url}"))

    sections = summary.strip().split("\n\n")

    for section in sections:
        lines = section.strip().split("\n")
        if not lines:
            continue

        heading = lines[0].strip().replace(":", "").upper()
        if use_typing:
            type_text(f"\n🟩 {heading}")
        else:
            display(Markdown(f"### 🟩 *{heading}*"))

        for line in lines[1:]:
            clean_line = line.strip().lstrip("*").strip()
            if clean_line:
                bullet = f"- {clean_line}"
                if use_typing:
                    type_text(bullet)
                else:
                    display(Markdown(bullet))


In [ ]:
url = input("Enter the URL of the website: ").strip()
print("\nFetching content and generating summary... Please wait.\n")

try:
    text = extract_text_from_url(url)
    summary = summarize_with_gemini(text)
    pretty_display(summary, url, use_typing=True)
except Exception as e:
    print("❌ Error fetching or summarizing content:", e)

In [ ]:
import os

In [ ]:
def save_summary_to_markdown(summary, filename="summary.md"):
    with open(filename, "w", encoding="utf-8") as md_file:
        md_file.write("# Website Summary\n\n")
        md_file.write(summary)
    print("✅ Summary saved at:", os.path.abspath("summary.md"))

In [ ]:
save_summary_to_markdown(summary)

In [ ]:
def search_keyword_in_summary(keyword, filename="summary.md"):
    keyword = keyword.lower()
    found_lines = []

    try:
        with open(filename, "r", encoding="utf-8") as md_file:
            lines = md_file.readlines()
            for line in lines:
                if keyword in line.lower():
                    found_lines.append(line.strip())

        if found_lines:
            print(f"\n🔍 Details for keyword: *{keyword}*\n")
            for match in found_lines:
                print("•", match)
        else:
            print(f"❌ The keyword '{keyword}' was not found in the summary.")
    
    except FileNotFoundError:
        print("⚠ Summary file not found. Make sure it's saved correctly.")

user_keyword = input("Enter a keyword to search in the summary: ")
search_keyword_in_summary(user_keyword)

In [ ]:
import json
import os
from datetime import datetime

def log_summary(source_url, summary_file_path, summary_text=None, model_used="Gemini"):
    history_file = "history.json"

    if os.path.exists(history_file):
        try:
            with open(history_file, "r") as f:
                history = json.load(f)
            if not isinstance(history, dict):
                history = {}
        except json.JSONDecodeError:
            history = {}
    else:
        history = {}

    summary_entry = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "source_url": source_url,
        "summary_file": os.path.basename(summary_file_path),
        "used_model": model_used
    }

    if summary_text:
        summary_entry["summary_preview"] = summary_text[:200] + "..."

    history.setdefault("summaries", []).append(summary_entry)

    with open(history_file, "w") as f:
        json.dump(history, f, indent=4)

    print(f"✅ Summary logged to '{history_file}'")


log_summary(url, "summary.md")